## Load packages

In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import csv

## Load data

In [4]:
# Load matrices from CSV files
ISC_EEG = np.genfromtxt(os.path.join('ISC_EEG.csv'),delimiter=',')
ISC_EEG = np.array(ISC_EEG,dtype='float') # Convert into array

ISC_EDA = np.genfromtxt(os.path.join('ISC_EDA.csv'),delimiter=',')
ISC_EDA = np.array(ISC_EDA,dtype='float') # Convert into array

ISC_IBI = np.genfromtxt(os.path.join('ISC_IBI.csv'),delimiter=',')
ISC_IBI = np.array(ISC_IBI,dtype='float') # Convert into array

subjectsWithoutShift = np.arange(len(ISC_EEG))

# Remove Inf in data for EDA and IBI (try to have similar ratio as in EEG between diagonal and mean non-diagonal)
tmp_EEG = np.copy(ISC_EEG)
tmp_EDA = np.copy(ISC_EDA)
tmp_IBI = np.copy(ISC_IBI)

for i in range(len(tmp_EEG)):
    tmp_EEG[i,i] = 0
    tmp_EDA[i,i] = 0
    tmp_IBI[i,i] = 0
eegNorm = 3/np.mean(tmp_EEG)

for i in range(len(ISC_IBI)):
    ISC_IBI[i,i] = np.mean(tmp_IBI)*eegNorm
    ISC_EDA[i,i] = np.mean(tmp_EDA)*eegNorm
    
# Load true clusters
def trueGroups():
    condition = np.genfromtxt(os.path.join('condition.csv'),delimiter=',')
    condition = np.array(condition,dtype='int') # Convert into array
    
    global subjectsWithoutShift
    subjectsWithoutShift = np.arange(len(condition))
    return condition

condition = trueGroups()
subjects = np.arange(len(condition))
narrative = np.where(condition == 0)[0]
stimulus = np.where(condition == 1)[0]

## Parameters

In [7]:
case = 'EEG'
nbParticipants = len(ISC_EEG) - 2

In [9]:
# Choose study matrix according to case study
if (case == 'EEG'):
    study_matrix = ISC_EEG
elif (case == 'EDA'):
    study_matrix = ISC_EDA
elif (case == 'IBI'):
    study_matrix = ISC_IBI
    
# Check if number of participants is valid, otherwise throw error
if nbParticipants>len(ISC_EEG):
    raise ValueError('Number of selected participants greater than total number of participants')

## Compute distance matrix

In [10]:
def computeDistanceMatrix(study_matrix=study_matrix):
    
    # Normalise maximum value to 1
    matrix_norm = np.copy(study_matrix)
    matrix_norm = matrix_norm / np.max(abs(matrix_norm))

    # Convert into distance matrix
    distance_matrix = np.sqrt((1-matrix_norm)) # Formula in Matlab and in Scikit to convert 
    
    # Interval MDS normalization
    a = np.min(np.sort(distance_matrix,axis=0)[1,:])
    distance_matrix = distance_matrix - a
    distance_matrix = distance_matrix - np.diag(np.diag(distance_matrix))
    
    # To make sure output is perfectly symmetrical
    return (distance_matrix+distance_matrix.T)/2

## Plot points

In [ ]:
from sklearn.decomposition import PCA

# Show MDS scaling
import matplotlib.patches as mpatches

def plotPoints(points,projection=False,title='Ground truth',subjects=subjects,subjectsWithoutShift=subjectsWithoutShift):
    
    condition = trueGroups(removeOutlier)  
    condition = condition[subjectsWithoutShift]
    subjects=np.arange(len(condition))
    NA = np.where(condition[subjects]==0)[0]
    SSA = np.where(condition[subjects]==1)[0]

    if (points.shape[1] != 1):
        ax = plt.axes([1.2,0,1.2,1.2])
        ax.set_aspect(aspect='equal')
        if (projection):
            points = transformPoints(points)

        for i in range(len(points)):
            ax.annotate(subjectsWithoutShift[i],(points[i,0],points[i,1]),xytext=(points[i,0]+(np.max(points[:,0])-np.min(points[:,0]))/50,points[i,1]))

        plt.scatter(points[NA,0],points[NA,1],color='blue',s=145,label='NA')
        plt.scatter(points[SSA,0],points[SSA,1],color='red',s=145,label='SSA')
    else:
        plt.scatter(points[NA],np.zeros((len(NA),1)),marker='x',color='blue')
        plt.scatter(points[SSA],np.zeros((len(SSA),1)),marker='x',color='red')
        orderedSubjectPlot = np.argsort(points.T)
        for i in range(len(points)):
            subj = orderedSubjectPlot[0,i]
            plt.annotate(subj,(points[subj,0],0),xytext=(points[subj,0]-((i+1)%2+1)*np.max(points)/35,0.01*(i%2)-0.0065))
        
    NA = mpatches.Patch(color='blue', label='NA')
    SSA = mpatches.Patch(color='red', label='SSA')
    plt.legend(handles=[NA,SSA])
    
    plt.title(title)
    
# Shepard diagram
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import euclidean_distances

def shepardDiagram(distance_matrix,points,disparities,title=''):
    ax = plt.axes([0,0,1.2,1.2])
    #ax.set_aspect(aspect='equal')
    
    order = np.lexsort((np.ravel(distance_matrix),np.ravel(disparities)))

    computedDistances = pdist(points);
    computedDistances = squareform(computedDistances);
    computedDistances = euclidean_distances(points)

    plt.scatter(np.ravel(distance_matrix),np.ravel(computedDistances))
    plt.plot(np.ravel(distance_matrix)[order],np.ravel(disparities)[order],'r.')
    plt.xlabel('Dissimilarities')
    plt.ylabel('Distances/Disparities')
    plt.title(title)
    corr = np.corrcoef(np.ravel(computedDistances),np.ravel(disparities))
    return corr[1,0]

## PCoA

In [11]:
from skbio.stats.ordination import pcoa

def computePCoACoordinate(distance_matrix,n_comp=2):
    
    pcoa_map = pcoa(distance_matrix, method='eigh', number_of_dimensions=n_comp)
    points = np.array(pcoa_map.samples)
    return points

## Multiview mapping

In [ ]:
ISC_EEG = np.genfromtxt(os.path.join('ISC_EEG.csv'),delimiter=',')
ISC_EEG = np.array(ISC_EEG,dtype='float') # Convert into array
ISC_EDA = np.genfromtxt(os.path.join('ISC_EDA.csv'),delimiter=',')
ISC_EDA = np.array(ISC_EDA,dtype='float') # Convert into array
ISC_IBI = np.genfromtxt(os.path.join('ISC_IBI.csv'),delimiter=',')
ISC_IBI = np.array(ISC_IBI,dtype='float') # Convert into array
subjectsWithoutShift = np.arange(len(ISC_EEG))

# Remove Inf in data for EDA and IBI (replace by sort of normalisation for the moment)
tmp_EEG = np.copy(ISC_EEG)
tmp_EDA = np.copy(ISC_EDA)
tmp_IBI = np.copy(ISC_IBI)

for i in range(len(tmp_EEG)):
    tmp_EEG[i,i] = 0
    tmp_EDA[i,i] = 0
    tmp_IBI[i,i] = 0
eegNorm = 3/np.mean(tmp_EEG)

for i in range(len(ISC_IBI)):
    ISC_IBI[i,i] = np.mean(tmp_IBI)*eegNorm
    ISC_EDA[i,i] = np.mean(tmp_EDA)*eegNorm

condition = trueGroups()

narrative = np.where(condition == 0)[0]
stimulus = np.where(condition == 1)[0]

study_matrix = ISC_EEG
N = len(study_matrix)
subjects = np.arange(N)
print(condition[subjects])

In [ ]:
def computeAllDistanceMatrix():
    sortedIdx = np.arange(N)

    #np.random.shuffle(sortedIdx)  
    #print(sortedIdx)
    
    mat_EEG = np.copy(ISC_EEG[sortedIdx,:])
    mat_EEG = mat_EEG[:,sortedIdx]
    mat_EDA = np.copy(ISC_EDA[sortedIdx,:])
    mat_EDA = mat_EDA[:,sortedIdx]
    mat_IBI = np.copy(ISC_IBI[sortedIdx,:])
    mat_IBI = mat_IBI[:,sortedIdx]

    global subjectsWithoutShift
    subjectsWithoutShift = sortedIdx[:p]
    idx = np.argsort(subjectsWithoutShift)
    subjectsWithoutShift = subjectsWithoutShift[idx]

    mat_EEG = mat_EEG[:p,:p]
    mat_EDA = mat_EDA[:p,:p]
    mat_IBI = mat_IBI[:p,:p]
    mat_EEG = mat_EEG[idx,:]
    mat_EDA = mat_EDA[idx,:]
    mat_IBI = mat_IBI[idx,:]
    mat_EEG = mat_EEG[:,idx]
    mat_EDA = mat_EDA[:,idx]
    mat_IBI = mat_IBI[:,idx]

    global subjects
    subjects = np.arange(p)
    condition = condition[subjectsWithoutShift]

    # Compute 3 distance matrices (according to each modality)
    distance_matrix_EEG = computeDistanceMatrix(mat_EEG)
    distance_matrix_EDA = computeDistanceMatrix(mat_EDA)
    distance_matrix_IBI = computeDistanceMatrix(mat_IBI)
  
    # Remove direct correlation influence and compute correlation distance matrices
    for i in range(len(mat_EEG)):
        mat_EEG[i,i] = 0
        mat_EDA[i,i] = 0
        mat_IBI[i,i] = 0
    distance_matrix_corr_EEG = pairwise_distances(mat_EEG,metric='correlation')
    distance_matrix_corr_EDA = pairwise_distances(mat_EDA,metric='correlation')
    distance_matrix_corr_IBI = pairwise_distances(mat_IBI,metric='correlation')

    # Return distance matrices
    return distance_matrix_EEG, distance_matrix_EDA, distance_matrix_IBI, distance_matrix_corr_EEG, distance_matrix_corr_EDA, distance_matrix_corr_IBI